# Setup

In [1]:
# disable debugging logs
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [2]:
# import all libraries before continuing
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
from tensorflow.keras.losses import Loss
from tensorflow.keras import layers

# Exercise 1
?? Todo: is this enough work?

In [3]:
df = pd.read_csv('winequality-white.csv')

# one hot encoding
one_hot_encoded_data = pd.get_dummies(df, columns = ['quality'])
one_hot_encoded_data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,False,False,False,True,False,False,False
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,False,False,False,True,False,False,False
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,False,False,False,True,False,False,False
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,False,False,False,True,False,False,False
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,False,False,False,True,False,False,False
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,False,False,True,False,False,False,False
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,False,False,False,True,False,False,False
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,False,False,False,False,True,False,False


In [4]:
# normalize each field
columns_to_normalize = df.columns.difference(['quality'])
output_columns = one_hot_encoded_data.columns.difference(columns_to_normalize)

scaler = MinMaxScaler()
one_hot_encoded_data[columns_to_normalize] = scaler.fit_transform(one_hot_encoded_data[columns_to_normalize])
one_hot_encoded_data[output_columns] = one_hot_encoded_data[output_columns].astype(int)
one_hot_encoded_data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9
0,0.307692,0.186275,0.216867,0.308282,0.106825,0.149826,0.373550,0.267785,0.254545,0.267442,0.129032,0,0,0,1,0,0,0
1,0.240385,0.215686,0.204819,0.015337,0.118694,0.041812,0.285383,0.132832,0.527273,0.313953,0.241935,0,0,0,1,0,0,0
2,0.413462,0.196078,0.240964,0.096626,0.121662,0.097561,0.204176,0.154039,0.490909,0.255814,0.338710,0,0,0,1,0,0,0
3,0.326923,0.147059,0.192771,0.121166,0.145401,0.156794,0.410673,0.163678,0.427273,0.209302,0.306452,0,0,0,1,0,0,0
4,0.326923,0.147059,0.192771,0.121166,0.145401,0.156794,0.410673,0.163678,0.427273,0.209302,0.306452,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,0.230769,0.127451,0.174699,0.015337,0.089021,0.076655,0.192575,0.077694,0.500000,0.325581,0.516129,0,0,0,1,0,0,0
4894,0.269231,0.235294,0.216867,0.113497,0.112760,0.191638,0.368910,0.150183,0.390909,0.279070,0.258065,0,0,1,0,0,0,0
4895,0.259615,0.156863,0.114458,0.009202,0.094955,0.097561,0.236659,0.104685,0.245455,0.279070,0.225806,0,0,0,1,0,0,0
4896,0.163462,0.205882,0.180723,0.007669,0.038576,0.062718,0.234339,0.030461,0.563636,0.186047,0.774194,0,0,0,0,1,0,0


# Exercise 2

$\frac{\partial L}{\partial w_{ko}} = \frac{\partial L}{\partial \hat y} * \frac{\partial \hat y}{\partial z_o} * \frac{\partial z_o}{\partial w_{ko}}$

$\frac{\partial L}{\partial w_{ko}} = -(y - \hat y) * \sigma(z_o)(1 - \sigma(z_o)) * x_k$

$\frac{\partial L}{\partial w_{ko}} = -(y - \hat y) * \hat y(1 - \hat y) * x_k$

$\frac{\partial L}{\partial w_{jk}} = \frac{\partial L}{\partial \hat y} * \frac{\partial \hat y}{\partial z_o} *\frac{\partial z_o}{\partial x_k} * \frac{\partial x_k}{\partial z_k} * \frac{\partial z_k}{\partial w_{jk}}$

$\frac{\partial L}{\partial w_{jk}} = -(y - \hat y) * \sigma(z_o)(1 - \sigma(z_o)) * w_{ko} * \sigma(z_k)(1 - \sigma(z_k)) * x_j$

$\frac{\partial L}{\partial w_{ij}} = \frac{\partial L}{\partial \hat y} * \frac{\partial \hat y}{\partial z_o} * (\frac{\partial z_o}{\partial x_{k1}} * \frac{\partial x_{k1}}{\partial z_{k1}} * \frac{\partial z_{k1}}{\partial x_j} + \frac{\partial z_o}{\partial x_{k2}} * \frac{\partial x_{k2}}{\partial z_{k2}} * \frac{\partial z_{k2}}{\partial x_j}) * \frac{\partial x_j}{\partial z_j} * \frac{\partial z_j}{\partial w_{ij}}$

$\frac{\partial L}{\partial w_{ij}} = -(y - \hat y) * \sigma(z_o)(1 - \sigma(z_o)) * (w_{ko1} * \sigma(z_{k1})(1 - \sigma(z_{k1})) * x_j + w_{ko2} * \sigma(z_{k2})(1 - \sigma(z_{k2})) * x_j) * 1 * x_i$

$w_{ko}^{t+1} = w_{ko}^t - \lambda * \frac{\partial L}{\partial w_{ko}}$

$w_{jk}^{t+1} = w_{jk}^t - \lambda * \frac{\partial L}{\partial w_{jk}}$

$w_{ij}^{t+1} = w_{ij}^t - \lambda * \frac{\partial L}{\partial w_{ij}}$

# Exercise 3

In [5]:
# Preprocess the data
X = one_hot_encoded_data[columns_to_normalize]
y = one_hot_encoded_data[output_columns]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

input_size = columns_to_normalize.shape
num_classes = output_columns.shape[0]

def createAndCompileModel():
    # Create neural network model
    model = tf.keras.Sequential([
        layers.Dense(23, activation='sigmoid', input_shape=input_size),
        layers.Dense(17, activation='sigmoid'),
        layers.Dense(13, activation='sigmoid'),
        layers.Dense(num_classes, activation='sigmoid')
    ])
    
    # Compile model with SGD and our SSE loss function
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.4)
    
    model.compile(optimizer=optimizer,
                  loss='mean_squared_error',
                  metrics=['accuracy'])

    return model

In [12]:
model = createAndCompileModel()

# Train the model
model.fit(X_train, y_train, epochs=500, batch_size=128)

Epoch 1/500
35/35 [==============================] - 1s 6ms/step - loss: 0.1895 - accuracy: 0.2334
Epoch 2/500
35/35 [==============================] - 0s 5ms/step - loss: 0.1164 - accuracy: 0.4181
Epoch 3/500
35/35 [==============================] - 0s 5ms/step - loss: 0.1048 - accuracy: 0.4499
Epoch 4/500
35/35 [==============================] - 0s 6ms/step - loss: 0.1012 - accuracy: 0.4499
Epoch 5/500
35/35 [==============================] - 0s 7ms/step - loss: 0.0997 - accuracy: 0.4499
Epoch 6/500
35/35 [==============================] - 0s 7ms/step - loss: 0.0988 - accuracy: 0.4499
Epoch 7/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0983 - accuracy: 0.4499
Epoch 8/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0979 - accuracy: 0.4499
Epoch 9/500
35/35 [==============================] - 0s 8ms/step - loss: 0.0977 - accuracy: 0.4499
Epoch 10/500
35/35 [==============================] - 0s 7ms/step - loss: 0.0975 - accuracy: 0.4499
Epoch 11/

In [13]:
# Predict the test data
y_pred = model.predict(X_test)

# Round classifications based on threshold of 0.5
y_pred = tf.round(y_pred)

# Create confusion matrix
class_names = y_test.columns
y_test_numpy = y_test.values
print("Structure for confusion matrix:")
print("[[TN FP]")
print(" [FN TP]]")
for i in range(num_classes):
    print("Confusion matrix for", class_names[i])
    cm = confusion_matrix(y_test_numpy[:, i], y_pred[:, i])
    print(cm)

# Calculate mean squared error
mse = model.evaluate(X_test, y_test)[0]
print("MSE =", mse)

16/16 [==============================] - 0s 4ms/step
Structure for confusion matrix:
[[TN FP]
 [FN TP]]
Confusion matrix for quality_3
[[487   0]
 [  3   0]]
Confusion matrix for quality_4
[[472   0]
 [ 18   0]]
Confusion matrix for quality_5
[[346   0]
 [144   0]]
Confusion matrix for quality_6
[[275   0]
 [215   0]]
Confusion matrix for quality_7
[[396   0]
 [ 94   0]]
Confusion matrix for quality_8
[[474   0]
 [ 16   0]]
Confusion matrix for quality_9
[[490]]
16/16 [==============================] - 0s 5ms/step - loss: 0.0974 - accuracy: 0.4388
MSE = 0.09738529473543167


# Exercise 4

In [14]:
# Do k-fold cross validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
mses = []
for i, (train_index, test_index) in enumerate(kfold.split(X)):    
    # Split the data according to the fold 
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]

    # Create and compile the same model from exercise 3
    model = createAndCompileModel()

    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=128)

    # Calculate mean squared error
    mses.append(model.evaluate(X_test, y_test)[0])

Epoch 1/100
35/35 [==============================] - 1s 6ms/step - loss: 0.1526 - accuracy: 0.3067
Epoch 2/100
35/35 [==============================] - 0s 6ms/step - loss: 0.1105 - accuracy: 0.4499
Epoch 3/100
35/35 [==============================] - 0s 6ms/step - loss: 0.1030 - accuracy: 0.4499
Epoch 4/100
35/35 [==============================] - 0s 6ms/step - loss: 0.1004 - accuracy: 0.4499
Epoch 5/100
35/35 [==============================] - 0s 6ms/step - loss: 0.0992 - accuracy: 0.4499
Epoch 6/100
35/35 [==============================] - 0s 7ms/step - loss: 0.0985 - accuracy: 0.4499
Epoch 7/100
35/35 [==============================] - 0s 6ms/step - loss: 0.0980 - accuracy: 0.4499
Epoch 8/100
35/35 [==============================] - 0s 6ms/step - loss: 0.0977 - accuracy: 0.4499
Epoch 9/100
35/35 [==============================] - 0s 6ms/step - loss: 0.0975 - accuracy: 0.4499
Epoch 10/100
35/35 [==============================] - 0s 6ms/step - loss: 0.0973 - accuracy: 0.4499
Epoch 11/

In [ ]:
# Print all the mses
for i, mse in enumerate(mses):
    print("MSE on iteration", i, "=", mse)

overall_avg_mse = np.mean(mses)
print("Overall average MSE =", overall_avg_mse)

# Exercise 5

In [6]:
# Define the parameter grid
parameter_grid = {
    'hidden_layer_sizes': [(10,), (50,), (100,), (10,10), (50,50), (100,100)],
    'learning_rate_init': [0.001, 0.01, 0.1],
    'max_iter': [100, 200, 300]
}

# Initialize the MLPRegressor
mlp = MLPRegressor()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=mlp, param_grid=parameter_grid, scoring='neg_mean_squared_error', cv=5)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best MSE:", abs(best_score))

/home/random/anaconda3/envs/ecs171/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/random/anaconda3/envs/ecs171/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/random/anaconda3/envs/ecs171/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/random/anaconda3/envs/ecs171/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  war

Best Parameters: {'hidden_layer_sizes': (100, 100), 'learning_rate_init': 0.001, 'max_iter': 100}
Best MSE: -0.08218024370202477
